In [1]:
from datetime import datetime 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import time
import random 

In [9]:
# The data imported from InfluxDB had unixtime in seconds. This cell turns this time into a human-readable date according to the local time. And exports to a .csv file

#Import data from CSV
data = pd.read_csv("./dataOpenmote.csv")
 
# Convert unixtime(in seconds) to local time
data['time'] = pd.DatetimeIndex(pd.to_datetime(data['time'], unit='s', utc=True)).tz_convert("America/Recife")

# During the experiment we had problems with some Rx. This list contains the time periods in which the Rx functioned as it should
intervalList = ["2020-11-24 11:30:00-03:00",
                "2020-11-25 23:15:12-03:00",
                "2020-11-26 07:34:12-03:00",
                "2020-11-26 22:23:41-03:00",
                "2020-11-26 23:16:33-03:00",
                "2020-11-29 12:46:09-03:00",
                "2020-11-29 17:14:34-03:00",
                "2020-11-29 18:00:00-03:00"]

openmoteData = pd.concat([data[data["time"] >= intervalList[0]][data["time"] <= intervalList[1]],
                          data[data["time"] >= intervalList[2]][data["time"] <= intervalList[3]],
                          data[data["time"] >= intervalList[4]][data["time"] <= intervalList[5]],
                          data[data["time"] >= intervalList[6]][data["time"] <= intervalList[7]]]).reset_index().drop(columns=["name", "index"])

infoNodeA = openmoteData[openmoteData["node_id"] == 'a']
infoNodeB = openmoteData[openmoteData["node_id"] == 'b']
infoNodeC = openmoteData[openmoteData["node_id"] == 'c']
infoNodeD = openmoteData[openmoteData["node_id"] == 'd']
infoNodeE = openmoteData[openmoteData["node_id"] == 'e']
infoNodeF = openmoteData[openmoteData["node_id"] == 'f']
infoNodeG = openmoteData[openmoteData["node_id"] == 'g']
infoNodeH = openmoteData[openmoteData["node_id"] == 'h']
infoNodeI = openmoteData[openmoteData["node_id"] == 'i']
infoNodeJ = openmoteData[openmoteData["node_id"] == 'j']
infoNodeK = openmoteData[openmoteData["node_id"] == 'k']

# Export openmoteData to csv with datetime
openmoteData.to_csv("./data/openmoteData.csv", index=False)
infoNodeA.to_csv("./data/infoNodeA.csv", index=False)
infoNodeB.to_csv("./data/infoNodeB.csv", index=False)
infoNodeC.to_csv("./data/infoNodeC.csv", index=False)
infoNodeD.to_csv("./data/infoNodeD.csv", index=False)
infoNodeE.to_csv("./data/infoNodeE.csv", index=False)
infoNodeF.to_csv("./data/infoNodeF.csv", index=False)
infoNodeG.to_csv("./data/infoNodeG.csv", index=False)
infoNodeH.to_csv("./data/infoNodeH.csv", index=False)
infoNodeI.to_csv("./data/infoNodeI.csv", index=False)
infoNodeJ.to_csv("./data/infoNodeJ.csv", index=False)
infoNodeK.to_csv("./data/infoNodeK.csv", index=False)

In [10]:
# Import data from .csv file
openmoteData = pd.read_csv("./data/openmoteData.csv")
openmoteData

,time,cca_retries,cca_rssi,modulation,node_id,pkt_number,pkt_retries,rssi
0,2020-11-24 11:30:01-03:00,1,-116,0,k,29,0,-109
1,2020-11-24 11:30:01-03:00,1,-119,1,k,29,0,-101
2,2020-11-24 11:30:01-03:00,1,-125,2,k,29,0,-91
3,2020-11-24 11:30:02-03:00,1,-115,0,k,29,1,-109
4,2020-11-24 11:30:02-03:00,1,-122,1,k,29,1,-100
...,...,...,...,...,...,...,...,...
462059,2020-11-29 17:59:43-03:00,1,-119,1,h,7718,2,-108
462060,2020-11-29 17:59:43-03:00,1,-125,2,h,7718,2,-94
462061,2020-11-29 17:59:50-03:00,1,-116,2,g,7706,0,-106
462062,2020-11-29 17:59:50-03:00,1,-110,2,g,7706,1,-105


In [6]:
# # Creates a dataframe for each node
nodeA = pd.read_csv("./data/nodeA.csv")
nodeB = pd.read_csv("./data/nodeB.csv")
nodeC = pd.read_csv("./data/nodeC.csv")
nodeD = pd.read_csv("./data/nodeD.csv")
nodeE = pd.read_csv("./data/nodeE.csv")
nodeF = pd.read_csv("./data/nodeF.csv")
nodeG = pd.read_csv("./data/nodeG.csv")
nodeH = pd.read_csv("./data/nodeH.csv")
nodeI = pd.read_csv("./data/nodeI.csv")
nodeJ = pd.read_csv("./data/nodeJ.csv")
nodeK = pd.read_csv("./data/nodeK.csv")

nodes = {"A": nodeA,
         "B": nodeB,
         "C": nodeC,
         "D": nodeD,
         "E": nodeE,
         "F": nodeF,
         "G": nodeG,
         "H": nodeH,
         "I": nodeI,
         "J": nodeJ,
         "K": nodeK}

nodeList = [nodeA, nodeB, nodeC, nodeD, nodeE, nodeF, nodeG, nodeH, nodeI, nodeJ, nodeK]

In [11]:
# # This cell generates the info_packet file for each node
for node in nodeList:
    packetList = []
    packetsLostList = []
    packetsReceived, packetsLost = 0, 0
    
    print("Node:", str(node['node_id'].iloc[0]).upper())

    for i in range(0, len(intervalList), 2):
        data = node[node["time"] >= intervalList[i]][node["time"] <= intervalList[i+1]]

        firstPacket, lastPacket = data["pkt_number"].min(), data["pkt_number"].max()
        for i in range(firstPacket, lastPacket+1):
            packet = node[node["pkt_number"] == i]
            
            
            if not packet.empty:
                packetList.append(packet)
                packetsReceived += 1
            else: 
                packetsLost += 1
                packetsLostList.append(i)

    
    pdr = packetsReceived/(packetsLost+packetsReceived)*100
    print(f'\tPackets Received: {packetsReceived}\n\tPackets Lost:{packetsLost}\n\tPDR: {pdr:.2f}')
    

    data = []
    
    # Make it true to generate info files about each node
    if True:
        for packet in packetList:
            packetInfo =  {
                'pkt_number': packet.iloc[0]["pkt_number"],
                'pkt_day': packet["time"].iloc[0],
                
                'fsk_0': 0,
                'fsk_1': 0,
                'fsk_2': 0,
                
                'oqpsk_0': 0,
                'oqpsk_1': 0,
                'oqpsk_2': 0,
                
                'ofdm_0': 0,
                'ofdm_1': 0,
                'ofdm_2': 0
                }

            # fsk
            if 0 in list(packet[packet['modulation'] == 0]['pkt_retries']): packetInfo["fsk_0"] = 1
            if 1 in list(packet[packet['modulation'] == 0]['pkt_retries']): packetInfo["fsk_1"] = 1
            if 2 in list(packet[packet['modulation'] == 0]['pkt_retries']): packetInfo["fsk_2"] = 1

            # oqpsk
            if 0 in list(packet[packet['modulation'] == 1]['pkt_retries']): packetInfo["oqpsk_0"] = 1
            if 1 in list(packet[packet['modulation'] == 1]['pkt_retries']): packetInfo["oqpsk_1"] = 1
            if 2 in list(packet[packet['modulation'] == 1]['pkt_retries']): packetInfo["oqpsk_2"] = 1

            # ofdm
            if 0 in list(packet[packet['modulation'] == 2]['pkt_retries']): packetInfo["ofdm_0"] = 1
            if 1 in list(packet[packet['modulation'] == 2]['pkt_retries']): packetInfo["ofdm_1"] = 1
            if 2 in list(packet[packet['modulation'] == 2]['pkt_retries']): packetInfo["ofdm_2"] = 1

            data.append(packetInfo)
            # print(packetInfo)
        
        pd.DataFrame(data).to_csv("./node_"+str(node['node_id'].iloc[0]).upper()+".csv", index=False)    

Node: A
	Packets Received: 6827
	Packets Lost:27
	PDR: 99.61
Node: B
	Packets Received: 5285
	Packets Lost:22
	PDR: 99.59
Node: C
	Packets Received: 6826
	Packets Lost:28
	PDR: 99.59
Node: D
	Packets Received: 6826
	Packets Lost:27
	PDR: 99.61
Node: E
	Packets Received: 6827
	Packets Lost:27
	PDR: 99.61
Node: F
	Packets Received: 6827
	Packets Lost:27
	PDR: 99.61
Node: G
	Packets Received: 5762
	Packets Lost:1091
	PDR: 84.08
Node: H
	Packets Received: 6826
	Packets Lost:27
	PDR: 99.61
Node: I
	Packets Received: 4533
	Packets Lost:2318
	PDR: 66.17
Node: J
	Packets Received: 2894
	Packets Lost:3948
	PDR: 42.30
Node: K
	Packets Received: 6820
	Packets Lost:35
	PDR: 99.49


In [7]:
# # This cell calculate some info about each node
for node, nodeData in nodes.items():
    packetsReceived, packetsLost = 0, 0
    
    print("Node:", node)
    for i in range(0, len(intervalList), 2):
        data = nodeData[nodeData["time"] >= intervalList[i]][nodeData["time"] <= intervalList[i+1]]
        
        firstPacket, lastPacket = data["pkt_number"].min(), data["pkt_number"].max()
        for p in range(firstPacket, lastPacket):
            a, _ = data[data["pkt_number"] == p].shape
            packetsReceived += a
            packetsLost += abs(9-a)
    totalPackets, _ = nodeData.shape
    pdr = packetsReceived/(packetsReceived+packetsLost)*100
    print(f'\tPackets Received: {packetsReceived}\n\tPackets Lost: {packetsLost}\n\tPHY PDR: {pdr:.1f}, Total Packets: {packetsReceived+packetsLost}')
    print(f'\tRSSI: {nodeData["rssi"].mean():.1f}\n\tCCA RSSI: {nodeData["cca_rssi"].mean()}')



Node: A
	Packets Received: 61628
	Packets Lost: 508
	PHY PDR: 99.2, Total Packets: 62136
	RSSI: -93.9
	RSSI: -106.10794346978558
Node: B
	Packets Received: 51815
	Packets Lost: 5210
	PHY PDR: 90.9, Total Packets: 57025
	RSSI: -96.4
	RSSI: -105.82600286532951
Node: C
	Packets Received: 45760
	Packets Lost: 16142
	PHY PDR: 73.9, Total Packets: 61902
	RSSI: -99.5
	RSSI: -115.2856015287471
Node: D
	Packets Received: 57599
	Packets Lost: 4458
	PHY PDR: 92.8, Total Packets: 62057
	RSSI: -95.3
	RSSI: -111.09147983517731
Node: E
	Packets Received: 55008
	Packets Lost: 7040
	PHY PDR: 88.7, Total Packets: 62048
	RSSI: -98.5
	RSSI: -109.91839285102887
Node: F
	Packets Received: 56461
	Packets Lost: 5597
	PHY PDR: 91.0, Total Packets: 62058
	RSSI: -98.8
	RSSI: -107.6774692899231
Node: G
	Packets Received: 15846
	Packets Lost: 45795
	PHY PDR: 25.7, Total Packets: 61641
	RSSI: -103.6
	RSSI: -113.80960484023444
Node: H
	Packets Received: 46511
	Packets Lost: 15378
	PHY PDR: 75.2, Total Packets: 61889

In [31]:
best_tx =  {"D": [1, 1, 1],
            "H": [3, 3, 1], 
            "A": [3, 1, 1],
            "B": [1, 1, 3],
            "C": [1, 3, 1],
            "E": [2, 1, 1],
            "F": [2, 1, 1],
            "G": [3, 1, 1],
            "I": [1, 1, 3],
            "J": [2, 1, 2],
            "K": [2, 1, 1]}

worse_tx = {"D": [1, 2, 1],
            "H": [1, 2, 1], 
            "A": [1, 2, 1],
            "B": [2, 2, 1],
            "C": [1, 2, 2],
            "E": [1, 2, 1],
            "F": [1, 2, 1],
            "G": [1, 1, 3],
            "I": [1, 2, 1],
            "J": [1, 1, 3],
            "K": [1, 2, 1]}

In [34]:
# # This cell calculates the PDR for each transmission attempt and generates some graphs.
# # This script was made by @peretuset

plt.rcParams.update({'errorbar.capsize': 2})

name  = "node_"
ext = '.csv'


nodes_1_flor = [ "D", "H" ]
nodes_2_flor = [ "A", "B", "C" ]
nodes_3_flor = [ "E", "F", "G" ]
nodes_4_flor = [ "I", "J", "K"]

nodes_all = nodes_1_flor + nodes_2_flor + nodes_3_flor + nodes_4_flor

fig_names = ["first_floor_ifpb", "second_floor_ifpb", "third_floor_ifpb", "fourth_floor_ifpb", "all_ifpb"]
node_names = [nodes_1_flor, nodes_2_flor, nodes_3_flor, nodes_4_flor, nodes_all]

for fig_name, nodes in zip(fig_names, node_names):
    best_tx =  {"D": [1, 1, 1],
                "H": [3, 3, 1], 
                "A": [3, 1, 1],
                "B": [1, 1, 3],
                "C": [1, 3, 1],
                "E": [2, 1, 1],
                "F": [2, 1, 1],
                "G": [3, 1, 1],
                "I": [1, 1, 3],
                "J": [2, 1, 2],
                "K": [2, 1, 1]}

    worse_tx = {"D": [1, 2, 1],
                "H": [1, 2, 1], 
                "A": [1, 2, 1],
                "B": [2, 2, 1],
                "C": [1, 2, 2],
                "E": [1, 2, 1],
                "F": [1, 2, 1],
                "G": [1, 1, 3],
                "I": [1, 2, 1],
                "J": [1, 1, 3],
                "K": [1, 2, 1]}
    only_fsk  = {new_list: [] for new_list in nodes}
    only_qpsk = {new_list: [] for new_list in nodes}
    only_ofdm = {new_list: [] for new_list in nodes}
    best      = {new_list: [] for new_list in nodes}
    worse     = {new_list: [] for new_list in nodes}

    fsk_mean  = []
    qpsk_mean = []
    ofdm_mean = []

    # Iterate over all nodes
    for node in nodes:
        print("Parsing node = {} ".format(node))

        # Create measurement name
        filename = name + node + ext

        # Read values from csv
        filename = 'node_' + node + '.csv'
        x = pd.read_csv(filename)

        x = x.set_index(["pkt_day", "pkt_number"])

        ##
        # Compute the PDR with FSK
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,0:3]
        for i in range(0,3):
            scratch = y[y.columns[i:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_fsk[node].append(pdr)
        print("\t FSK - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with OQPSK
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,3:6]
        for i in range(0,3):
            scratch = y[y.columns[i:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_qpsk[node].append(pdr)
        print("\t OQPSK - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with OFDM
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,6:9]
        for i in range(0,3):
            scratch = y[y.columns[i:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_ofdm[node].append(pdr)
        print("\t OFDM - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with BEST
        ##
        r = []
        y = pd.DataFrame()
        best_tx[node][0] += 0 - 1
        best_tx[node][1] += 3 - 1
        best_tx[node][2] += 6 - 1
        y[['x1', 'y1', 'z1']] = x.iloc[:,best_tx[node]]
        for i in range(0,3):
            scratch = y[y.columns[i:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            best[node].append(pdr)
        print("\t BEST - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with WORSE
        ##
        r = []
        y = pd.DataFrame()
        worse_tx[node][0] += 0 - 1
        worse_tx[node][1] += 3 - 1
        worse_tx[node][2] += 6 - 1
        y[['x1', 'y1', 'z1']] = x.iloc[:,worse_tx[node]]
        for i in range(0,3):
            scratch = y[y.columns[i:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            worse[node].append(pdr)
        print("\t WORSE - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

    print()

    # Convert ordered and unordered dictionaries to DataFrames
    fsk = pd.DataFrame(only_fsk).T
    qpsk = pd.DataFrame(only_qpsk).T
    ofdm = pd.DataFrame(only_ofdm).T
    best = pd.DataFrame(best).T
    worse = pd.DataFrame(worse).T

    # Compute mean and standard deviation
    fsk_mean = fsk.mean()
    fsk_max = fsk.max()
    fsk_min = fsk.min()
    qpsk_mean = qpsk.mean()
    qpsk_max = qpsk.max()
    qpsk_min = qpsk.min()
    ofdm_mean = ofdm.mean()
    ofdm_max = ofdm.max()
    ofdm_min = ofdm.min()
    best_mean = best.mean()
    best_max = best.max()
    best_min = best.min()
    worse_mean = worse.mean()
    worse_max = worse.max()
    worse_min = worse.min()

    # Print results to screen
    print("FSK MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_mean.T])))
    print("FSK MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_max.T])))
    print("FSK MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_min.T])))

    print("QPSK MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_mean.T])))
    print("QPSK MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_max.T])))
    print("QPSK MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_min.T])))

    print("OFDM MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_mean.T])))
    print("OFDM MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_max.T])))
    print("OFDM MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_min.T])))

    print("WORSE MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_mean.T])))
    print("WORSE MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_max.T])))
    print("WORSE MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_min.T])))

    print("BEST MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_mean.T])))
    print("BEST MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_max.T])))
    print("BEST MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_min.T])))

    labels = ['1', '2', '3']
    width = 0.3  # the width of the bars
    x = np.arange(len(labels))

    ax = plt.axes()

    # Create resulting plot
    ax.bar(x - 2*width/2, fsk_mean,    width/2, yerr=[fsk_mean-fsk_min,       fsk_max-fsk_mean],       label="Only FSK")
    ax.bar(x - 1*width/2, qpsk_mean,   width/2, yerr=[qpsk_mean-qpsk_min,     qpsk_max-qpsk_mean],     label="Only OQPSK")
    ax.bar(x - 0*width/2, ofdm_mean,   width/2, yerr=[ofdm_mean-ofdm_min,     ofdm_max-ofdm_mean],     label="Only OFDM")
    ax.bar(x + 1*width/2, worse_mean,  width/2, yerr=[worse_mean-worse_min,   worse_max-worse_mean],   label="Combined WORST")
    ax.bar(x + 2*width/2, best_mean,   width/2, yerr=[best_mean-best_min,     best_max-best_mean],     label="Combined BEST")

    ax.yaxis.set_minor_locator(plt.MultipleLocator(1))

    plt.grid(which='major', linestyle='-')
    plt.grid(which='minor', linestyle='--')
    #plt.title('Success probability with n transmission attemps')
    plt.xlabel('Transmission attempts')
    plt.ylabel('Success probability (%)')
    plt.xticks(x, labels)
    plt.ylim(0, 100)
    plt.legend(loc="lower right")
    plt.tight_layout()

    plt.savefig('./figures/pdr/mod_' + fig_name + '.png', dpi=600)
    
    plt.clf()
    
    print("****************************************")

Parsing node = D 
	 FSK - PDR = 99.6 & 99.6 & 99.6
	 OQPSK - PDR = 80.7 & 79.0 & 80.5
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BEST - PDR = 99.6 & 80.7 & 99.6
	 WORSE - PDR = 99.6 & 79.0 & 99.6
Parsing node = H 
	 FSK - PDR = 45.3 & 45.4 & 45.5
	 OQPSK - PDR = 81.2 & 79.0 & 81.3
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BEST - PDR = 45.5 & 81.3 & 99.6
	 WORSE - PDR = 45.3 & 79.0 & 99.6

FSK MEAN = 72.5 & 72.5 & 72.5
FSK MAX = 99.6 & 99.6 & 99.6
FSK MIN = 45.3 & 45.4 & 45.5
QPSK MEAN = 80.9 & 79.0 & 80.9
QPSK MAX = 81.2 & 79.0 & 81.3
QPSK MIN = 80.7 & 79.0 & 80.5
OFDM MEAN = 99.6 & 99.6 & 99.6
OFDM MAX = 99.6 & 99.6 & 99.6
OFDM MIN = 99.6 & 99.6 & 99.6
WORSE MEAN = 72.5 & 79.0 & 99.6
WORSE MAX = 99.6 & 79.0 & 99.6
WORSE MIN = 45.3 & 79.0 & 99.6
BEST MEAN = 72.5 & 81.0 & 99.6
BEST MAX = 99.6 & 81.3 & 99.6
BEST MIN = 45.5 & 80.7 & 99.6
****************************************
Parsing node = A 
	 FSK - PDR = 99.5 & 99.5 & 99.6
	 OQPSK - PDR = 99.6 & 99.5 & 99.5
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BES

<Figure size 432x288 with 0 Axes>

In [30]:
nodeDFSK    = [99.6, 99.6, 99.6]
nodeDOQPSK  = [80.7, 79.0, 80.5]
nodeDOFDM   = [99.6, 99.6, 99.6]

nodeHFSK    = [45.3, 45.4, 45.5]
nodeHOQPSK  = [81.2, 79.0, 81.3]
nodeHOFDM   = [99.6, 99.6, 99.6]

nodeAFSK    = [99.5, 99.5, 99.6]
nodeAOQPSK  = [99.6, 99.5, 99.5]
nodeAOFDM   = [99.6, 99.6, 99.6]

nodeBFSK    = [99.5, 99.4, 99.5]
nodeBOQPSK  = [98.5, 98.4, 98.5]
nodeBOFDM   = [98.3, 98.7, 98.8]

nodeCFSK    = [96.8, 96.8, 96.8]
nodeCOQPSK  = [28.2, 20.3, 28.3]
nodeCOFDM   = [99.5, 99.4, 99.4]

nodeEFSK    = [98.9, 99.0, 99.0]
nodeEOQPSK  = [70.0, 64.1, 70.0]
nodeEOFDM   = [99.6, 99.6, 99.6]

nodeFFSK    = [88.5, 88.6, 88.5]
nodeFOQPSK  = [86.9, 83.7, 86.0]
nodeFOFDM   = [99.6, 99.6, 99.6]

nodeGFSK    = [00.1, 00.1, 00.2]
nodeGOQPSK  = [00.2, 00.2, 00.2]
nodeGOFDM   = [91.1, 90.9, 90.2]

nodeIFSK    = [00.0, 00.0, 00.0]
nodeIOQPSK  = [09.2, 08.7, 09.2]
nodeIOFDM   = [87.5, 87.5, 88.2]

nodeJFSK    = [04.2, 04.6, 04.3]
nodeJOQPSK  = [00.0, 00.0, 00.0]
nodeJOFDM   = [59.2, 59.9, 58.8]

nodeKFSK    = [57.0, 57.6, 57.2]
nodeKOQPSK  = [98.1, 97.9, 98.0]
nodeKOFDM   = [98.7, 98.7, 98.7]


print(f'"D": [{nodeDFSK.index(max(nodeDFSK))+1}, {nodeDOQPSK.index(max(nodeDOQPSK))+1}, {nodeDOFDM.index(max(nodeDOFDM))+1}]')
print(f'"H": [{nodeHFSK.index(max(nodeHFSK))+1}, {nodeHOQPSK.index(max(nodeHOQPSK))+1}, {nodeHOFDM.index(max(nodeHOFDM))+1}]')
print(f'"A": [{nodeAFSK.index(max(nodeAFSK))+1}, {nodeAOQPSK.index(max(nodeAOQPSK))+1}, {nodeAOFDM.index(max(nodeAOFDM))+1}]')
print(f'"B": [{nodeBFSK.index(max(nodeBFSK))+1}, {nodeBOQPSK.index(max(nodeBOQPSK))+1}, {nodeBOFDM.index(max(nodeBOFDM))+1}]')
print(f'"C": [{nodeCFSK.index(max(nodeCFSK))+1}, {nodeCOQPSK.index(max(nodeCOQPSK))+1}, {nodeCOFDM.index(max(nodeCOFDM))+1}]')
print(f'"E": [{nodeEFSK.index(max(nodeEFSK))+1}, {nodeEOQPSK.index(max(nodeEOQPSK))+1}, {nodeEOFDM.index(max(nodeEOFDM))+1}]')
print(f'"F": [{nodeFFSK.index(max(nodeFFSK))+1}, {nodeFOQPSK.index(max(nodeFOQPSK))+1}, {nodeFOFDM.index(max(nodeFOFDM))+1}]')
print(f'"G": [{nodeGFSK.index(max(nodeGFSK))+1}, {nodeGOQPSK.index(max(nodeGOQPSK))+1}, {nodeGOFDM.index(max(nodeGOFDM))+1}]')
print(f'"I": [{nodeIFSK.index(max(nodeIFSK))+1}, {nodeIOQPSK.index(max(nodeIOQPSK))+1}, {nodeIOFDM.index(max(nodeIOFDM))+1}]')
print(f'"J": [{nodeJFSK.index(max(nodeJFSK))+1}, {nodeJOQPSK.index(max(nodeJOQPSK))+1}, {nodeJOFDM.index(max(nodeJOFDM))+1}]')
print(f'"K": [{nodeKFSK.index(max(nodeKFSK))+1}, {nodeKOQPSK.index(max(nodeKOQPSK))+1}, {nodeKOFDM.index(max(nodeKOFDM))+1}]')
print()

print(f'"D": [{nodeDFSK.index(min(nodeDFSK))+1}, {nodeDOQPSK.index(min(nodeDOQPSK))+1}, {nodeDOFDM.index(min(nodeDOFDM))+1}]')
print(f'"H": [{nodeHFSK.index(min(nodeHFSK))+1}, {nodeHOQPSK.index(min(nodeHOQPSK))+1}, {nodeHOFDM.index(min(nodeHOFDM))+1}]')
print(f'"A": [{nodeAFSK.index(min(nodeAFSK))+1}, {nodeAOQPSK.index(min(nodeAOQPSK))+1}, {nodeAOFDM.index(min(nodeAOFDM))+1}]')
print(f'"B": [{nodeBFSK.index(min(nodeBFSK))+1}, {nodeBOQPSK.index(min(nodeBOQPSK))+1}, {nodeBOFDM.index(min(nodeBOFDM))+1}]')
print(f'"C": [{nodeCFSK.index(min(nodeCFSK))+1}, {nodeCOQPSK.index(min(nodeCOQPSK))+1}, {nodeCOFDM.index(min(nodeCOFDM))+1}]')
print(f'"E": [{nodeEFSK.index(min(nodeEFSK))+1}, {nodeEOQPSK.index(min(nodeEOQPSK))+1}, {nodeEOFDM.index(min(nodeEOFDM))+1}]')
print(f'"F": [{nodeFFSK.index(min(nodeFFSK))+1}, {nodeFOQPSK.index(min(nodeFOQPSK))+1}, {nodeFOFDM.index(min(nodeFOFDM))+1}]')
print(f'"G": [{nodeGFSK.index(min(nodeGFSK))+1}, {nodeGOQPSK.index(min(nodeGOQPSK))+1}, {nodeGOFDM.index(min(nodeGOFDM))+1}]')
print(f'"I": [{nodeIFSK.index(min(nodeIFSK))+1}, {nodeIOQPSK.index(min(nodeIOQPSK))+1}, {nodeIOFDM.index(min(nodeIOFDM))+1}]')
print(f'"J": [{nodeJFSK.index(min(nodeJFSK))+1}, {nodeJOQPSK.index(min(nodeJOQPSK))+1}, {nodeJOFDM.index(min(nodeJOFDM))+1}]')
print(f'"K": [{nodeKFSK.index(min(nodeKFSK))+1}, {nodeKOQPSK.index(min(nodeKOQPSK))+1}, {nodeKOFDM.index(min(nodeKOFDM))+1}]')

"D": [1, 1, 1]
"H": [3, 3, 1]
"A": [3, 1, 1]
"B": [1, 1, 3]
"C": [1, 3, 1]
"E": [2, 1, 1]
"F": [2, 1, 1]
"G": [3, 1, 1]
"I": [1, 1, 3]
"J": [2, 1, 2]
"K": [2, 1, 1]

"D": [1, 2, 1]
"H": [1, 2, 1]
"A": [1, 2, 1]
"B": [2, 2, 1]
"C": [1, 2, 2]
"E": [1, 2, 1]
"F": [1, 2, 1]
"G": [1, 1, 3]
"I": [1, 2, 1]
"J": [1, 1, 3]
"K": [1, 2, 1]


In [39]:
# # This cell calculates the PDR for accumulated attempts and generates some graphs.
# # This script was made by @peretuset

plt.rcParams.update({'errorbar.capsize': 2})

name  = "node_"
ext = '.csv'


nodes_1_flor = [ "D", "H" ]
nodes_2_flor = [ "A", "B", "C" ]
nodes_3_flor = [ "E", "F", "G" ]
nodes_4_flor = [ "I", "J", "K"]

nodes_all = nodes_1_flor + nodes_2_flor + nodes_3_flor + nodes_4_flor

fig_names = ["first_floor_ifpb_acumulated", "second_floor_ifpb_acumulated", "third_floor_ifpb_acumulated", "fourth_floor_ifpb_acumulated", "all_ifpb_acumulated"]
node_names = [nodes_1_flor, nodes_2_flor, nodes_3_flor, nodes_4_flor, nodes_all]

for fig_name, nodes in zip(fig_names, node_names):
    best_tx =  {"D": [1, 1, 1],
                "H": [1, 3, 3],
                "A": [1, 1, 3],
                "B": [3, 1, 1],
                "C": [1, 3, 1],
                "E": [1, 1, 2],
                "F": [1, 1, 2],
                "G": [1, 1, 3],
                "I": [3, 1, 1],
                "J": [2, 1, 2],
                "K": [1, 1, 2]}
    worse_tx = {"D": [1, 2, 1],
                "H": [1, 2, 1],
                "A": [1, 2, 1],
                "B": [1, 2, 2],
                "C": [2, 2, 1],
                "E": [1, 2, 1],
                "F": [1, 2, 1],
                "G": [3, 1, 1],
                "I": [1, 2, 1],
                "J": [3, 1, 1],
                "K": [1, 2, 1]}

    only_fsk  = {new_list: [] for new_list in nodes}
    only_qpsk = {new_list: [] for new_list in nodes}
    only_ofdm = {new_list: [] for new_list in nodes}
    best      = {new_list: [] for new_list in nodes}
    worse     = {new_list: [] for new_list in nodes}

    fsk_mean  = []
    qpsk_mean = []
    ofdm_mean = []

    # Iterate over all nodes
    for node in nodes:
        print("Parsing node = {} ".format(node))

        # Create measurement name
        filename = name + node + ext

        # Read values from csv
        filename = 'node_' + node + '.csv'
        x = pd.read_csv(filename)

        x = x.set_index(["pkt_day", "pkt_number"])

        ##
        # Compute the PDR with FSK
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,0:3]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_fsk[node].append(pdr)
        print("\t FSK - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with OQPSK
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,3:6]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_qpsk[node].append(pdr)
        print("\t OQPSK - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with OFDM
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,6:9]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_ofdm[node].append(pdr)
        print("\t OFDM - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with BEST
        ##
        r = []
        y = pd.DataFrame()
        best_tx[node][2] += 0 - 1
        best_tx[node][1] += 3 - 1
        best_tx[node][0] += 6 - 1
        y[['x1', 'y1', 'z1']] = x.iloc[:,best_tx[node]]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            best[node].append(pdr)
        print("\t BEST - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with WORSE
        ##
        r = []
        y = pd.DataFrame()
        worse_tx[node][2] += 0 - 1
        worse_tx[node][1] += 3 - 1
        worse_tx[node][0] += 6 - 1
        y[['x1', 'y1', 'z1']] = x.iloc[:,worse_tx[node]]
        for i in  range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            worse[node].append(pdr)
        print("\t WORSE - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

    print()

    # Convert ordered and unordered dictionaries to DataFrames
    fsk = pd.DataFrame(only_fsk).T
    qpsk = pd.DataFrame(only_qpsk).T
    ofdm = pd.DataFrame(only_ofdm).T
    best = pd.DataFrame(best).T
    worse = pd.DataFrame(worse).T

    # Compute mean and standard deviation
    fsk_mean = fsk.mean()
    fsk_max = fsk.max()
    fsk_min = fsk.min()
    qpsk_mean = qpsk.mean()
    qpsk_max = qpsk.max()
    qpsk_min = qpsk.min()
    ofdm_mean = ofdm.mean()
    ofdm_max = ofdm.max()
    ofdm_min = ofdm.min()
    best_mean = best.mean()
    best_max = best.max()
    best_min = best.min()
    worse_mean = worse.mean()
    worse_max = worse.max()
    worse_min = worse.min()

    # Print results to screen
    print("FSK MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_mean.T])))
    print("FSK MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_max.T])))
    print("FSK MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_min.T])))

    print("QPSK MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_mean.T])))
    print("QPSK MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_max.T])))
    print("QPSK MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_min.T])))

    print("OFDM MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_mean.T])))
    print("OFDM MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_max.T])))
    print("OFDM MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_min.T])))

    print("WORSE MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_mean.T])))
    print("WORSE MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_max.T])))
    print("WORSE MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_min.T])))

    print("BEST MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_mean.T])))
    print("BEST MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_max.T])))
    print("BEST MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_min.T])))

    labels = ['1', '2', '3']
    width = 0.3  # the width of the bars
    x = np.arange(len(labels))

    ax = plt.axes()

    # Create resulting plot
    ax.bar(x - 2*width/2, fsk_mean,    width/2, yerr=[fsk_mean-fsk_min,       fsk_max-fsk_mean],       label="Only FSK")
    ax.bar(x - 1*width/2, qpsk_mean,   width/2, yerr=[qpsk_mean-qpsk_min,     qpsk_max-qpsk_mean],     label="Only OQPSK")
    ax.bar(x - 0*width/2, ofdm_mean,   width/2, yerr=[ofdm_mean-ofdm_min,     ofdm_max-ofdm_mean],     label="Only OFDM")
    ax.bar(x + 1*width/2, worse_mean,  width/2, yerr=[worse_mean-worse_min,   worse_max-worse_mean],   label="Combined WORST")
    ax.bar(x + 2*width/2, best_mean,   width/2, yerr=[best_mean-best_min,     best_max-best_mean],     label="Combined BEST")

    ax.yaxis.set_minor_locator(plt.MultipleLocator(1))

    plt.grid(which='major', linestyle='-')
    plt.grid(which='minor', linestyle='--')
    #plt.title('Success probability with n transmission attemps')
    plt.xlabel('Transmission attempts')
    plt.ylabel('Success probability (%)')
    plt.xticks(x, labels)
    plt.ylim(0, 100)
    plt.legend(loc="lower right")
    plt.tight_layout()

    plt.savefig('./figures/acumulated_pdr/mod_' + fig_name + '.png', dpi=600)
    
    plt.clf()
    
    print("****************************************")

Parsing node = D 
	 FSK - PDR = 99.6 & 99.6 & 99.6
	 OQPSK - PDR = 80.7 & 81.8 & 83.5
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BEST - PDR = 99.6 & 99.6 & 99.6
	 WORSE - PDR = 99.6 & 99.6 & 99.6
Parsing node = H 
	 FSK - PDR = 45.3 & 47.0 & 48.4
	 OQPSK - PDR = 81.2 & 83.1 & 85.9
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BEST - PDR = 99.6 & 99.6 & 99.6
	 WORSE - PDR = 99.6 & 99.6 & 99.6

FSK MEAN = 72.5 & 73.3 & 74.0
FSK MAX = 99.6 & 99.6 & 99.6
FSK MIN = 45.3 & 47.0 & 48.4
QPSK MEAN = 80.9 & 82.4 & 84.7
QPSK MAX = 81.2 & 83.1 & 85.9
QPSK MIN = 80.7 & 81.8 & 83.5
OFDM MEAN = 99.6 & 99.6 & 99.6
OFDM MAX = 99.6 & 99.6 & 99.6
OFDM MIN = 99.6 & 99.6 & 99.6
WORSE MEAN = 99.6 & 99.6 & 99.6
WORSE MAX = 99.6 & 99.6 & 99.6
WORSE MIN = 99.6 & 99.6 & 99.6
BEST MEAN = 99.6 & 99.6 & 99.6
BEST MAX = 99.6 & 99.6 & 99.6
BEST MIN = 99.6 & 99.6 & 99.6
****************************************
Parsing node = A 
	 FSK - PDR = 99.5 & 99.6 & 99.6
	 OQPSK - PDR = 99.6 & 99.6 & 99.6
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BES

<Figure size 432x288 with 0 Axes>

In [40]:
# # This cell calculates the PDR for accumulated attempts and generates some graphs. This graphs does not have best and worse bar plotted
# # This script was made by @peretuset

plt.rcParams.update({'errorbar.capsize': 2})

name  = "node_"
ext = '.csv'


nodes_1_flor = [ "D", "H" ]
nodes_2_flor = [ "A", "B", "C" ]
nodes_3_flor = [ "E", "F", "G" ]
nodes_4_flor = [ "I", "J", "K"]

nodes_all = nodes_1_flor + nodes_2_flor + nodes_3_flor + nodes_4_flor

fig_names = ["first_floor_ifpb_acumulated_wbw", "second_floor_ifpb_acumulated_wbw", "third_floor_ifpb_acumulated_wbw", "fourth_floor_ifpb_acumulated_wbw", "all_ifpb_acumulated_wbw"]
node_names = [nodes_1_flor, nodes_2_flor, nodes_3_flor, nodes_4_flor, nodes_all]

for fig_name, nodes in zip(fig_names, node_names):
    best_tx =  {"D": [1, 1, 1],
                "H": [1, 3, 3],
                "A": [1, 1, 3],
                "B": [3, 1, 1],
                "C": [1, 3, 1],
                "E": [1, 1, 2],
                "F": [1, 1, 2],
                "G": [1, 1, 3],
                "I": [3, 1, 1],
                "J": [2, 1, 2],
                "K": [1, 1, 2]}
    worse_tx = {"D": [1, 2, 1],
                "H": [1, 2, 1],
                "A": [1, 2, 1],
                "B": [1, 2, 2],
                "C": [2, 2, 1],
                "E": [1, 2, 1],
                "F": [1, 2, 1],
                "G": [3, 1, 1],
                "I": [1, 2, 1],
                "J": [3, 1, 1],
                "K": [1, 2, 1]}
                
    only_fsk  = {new_list: [] for new_list in nodes}
    only_qpsk = {new_list: [] for new_list in nodes}
    only_ofdm = {new_list: [] for new_list in nodes}
    best      = {new_list: [] for new_list in nodes}
    worse     = {new_list: [] for new_list in nodes}

    fsk_mean  = []
    qpsk_mean = []
    ofdm_mean = []

    # Iterate over all nodes
    for node in nodes:
        print("Parsing node = {} ".format(node))

        # Create measurement name
        filename = name + node + ext

        # Read values from csv
        filename = 'node_' + node + '.csv'
        x = pd.read_csv(filename)

        x = x.set_index(["pkt_day", "pkt_number"])

        ##
        # Compute the PDR with FSK
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,0:3]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_fsk[node].append(pdr)
        print("\t FSK - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with OQPSK
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,3:6]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_qpsk[node].append(pdr)
        print("\t OQPSK - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with OFDM
        ##
        r = []
        y = pd.DataFrame()
        y[['x1', 'y1', 'z1']] = x.iloc[:,6:9]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            only_ofdm[node].append(pdr)
        print("\t OFDM - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with BEST
        ##
        r = []
        y = pd.DataFrame()
        best_tx[node][2] += 0 - 1
        best_tx[node][1] += 3 - 1
        best_tx[node][0] += 6 - 1
        y[['x1', 'y1', 'z1']] = x.iloc[:,best_tx[node]]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            best[node].append(pdr)
        print("\t BEST - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

        ##
        # Compute the PDR with WORSE
        ##
        r = []
        y = pd.DataFrame()
        worse_tx[node][2] += 0 - 1
        worse_tx[node][1] += 3 - 1
        worse_tx[node][0] += 6 - 1
        y[['x1', 'y1', 'z1']] = x.iloc[:,worse_tx[node]]
        for i in range(0,3):
            scratch = y[y.columns[0:i+1]].sum(axis=1)
            z = (scratch > 0).astype(int)
            items = z.count()
            packets = z.sum()
            pdr = 100.0 * packets / items
            r.append(pdr)
            worse[node].append(pdr)
        print("\t WORSE - PDR = {:.1f} & {:.1f} & {:.1f}".format(r[0], r[1], r[2]))

    print()

    # Convert ordered and unordered dictionaries to DataFrames
    fsk = pd.DataFrame(only_fsk).T
    qpsk = pd.DataFrame(only_qpsk).T
    ofdm = pd.DataFrame(only_ofdm).T
    best = pd.DataFrame(best).T
    worse = pd.DataFrame(worse).T

    # Compute mean and standard deviation
    fsk_mean = fsk.mean()
    fsk_max = fsk.max()
    fsk_min = fsk.min()
    qpsk_mean = qpsk.mean()
    qpsk_max = qpsk.max()
    qpsk_min = qpsk.min()
    ofdm_mean = ofdm.mean()
    ofdm_max = ofdm.max()
    ofdm_min = ofdm.min()
    best_mean = best.mean()
    best_max = best.max()
    best_min = best.min()
    worse_mean = worse.mean()
    worse_max = worse.max()
    worse_min = worse.min()

    # Print results to screen
    print("FSK MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_mean.T])))
    print("FSK MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_max.T])))
    print("FSK MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in fsk_min.T])))

    print("QPSK MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_mean.T])))
    print("QPSK MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_max.T])))
    print("QPSK MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in qpsk_min.T])))

    print("OFDM MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_mean.T])))
    print("OFDM MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_max.T])))
    print("OFDM MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in ofdm_min.T])))

    print("WORSE MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_mean.T])))
    print("WORSE MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_max.T])))
    print("WORSE MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in worse_min.T])))

    print("BEST MEAN = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_mean.T])))
    print("BEST MAX = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_max.T])))
    print("BEST MIN = {}".format(' & '.join(['{:.1f}'.format(x) for x in best_min.T])))

    labels = ['1', '2', '3']
    width = 0.3  # the width of the bars
    x = np.arange(len(labels))

    ax = plt.axes()

    # Create resulting plot
    ax.bar(x - 2*width/2, fsk_mean,    width/2, yerr=[fsk_mean-fsk_min,       fsk_max-fsk_mean],       label="Only FSK")
    ax.bar(x - 1*width/2, qpsk_mean,   width/2, yerr=[qpsk_mean-qpsk_min,     qpsk_max-qpsk_mean],     label="Only OQPSK")
    ax.bar(x - 0*width/2, ofdm_mean,   width/2, yerr=[ofdm_mean-ofdm_min,     ofdm_max-ofdm_mean],     label="Only OFDM")
    # ax.bar(x + 1*width/2, worse_mean,  width/2, yerr=[worse_mean-worse_min,   worse_max-worse_mean],   label="Combined WORST")
    # ax.bar(x + 2*width/2, best_mean,   width/2, yerr=[best_mean-best_min,     best_max-best_mean],     label="Combined BEST")

    ax.yaxis.set_minor_locator(plt.MultipleLocator(1))

    plt.grid(which='major', linestyle='-')
    plt.grid(which='minor', linestyle='--')
    #plt.title('Success probability with n transmission attemps')
    plt.xlabel('Transmission attempts')
    plt.ylabel('Success probability (%)')
    plt.xticks(x, labels)
    plt.ylim(0, 100)
    plt.legend(loc="lower right")
    plt.tight_layout()

    plt.savefig('./figures/acumulated_pdr_wbw/mod' + fig_name + '.png', dpi=600)
    
    plt.clf()
    
    print("****************************************")

Parsing node = D 
	 FSK - PDR = 99.6 & 99.6 & 99.6
	 OQPSK - PDR = 80.7 & 81.8 & 83.5
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BEST - PDR = 99.6 & 99.6 & 99.6
	 WORSE - PDR = 99.6 & 99.6 & 99.6
Parsing node = H 
	 FSK - PDR = 45.3 & 47.0 & 48.4
	 OQPSK - PDR = 81.2 & 83.1 & 85.9
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BEST - PDR = 99.6 & 99.6 & 99.6
	 WORSE - PDR = 99.6 & 99.6 & 99.6

FSK MEAN = 72.5 & 73.3 & 74.0
FSK MAX = 99.6 & 99.6 & 99.6
FSK MIN = 45.3 & 47.0 & 48.4
QPSK MEAN = 80.9 & 82.4 & 84.7
QPSK MAX = 81.2 & 83.1 & 85.9
QPSK MIN = 80.7 & 81.8 & 83.5
OFDM MEAN = 99.6 & 99.6 & 99.6
OFDM MAX = 99.6 & 99.6 & 99.6
OFDM MIN = 99.6 & 99.6 & 99.6
WORSE MEAN = 99.6 & 99.6 & 99.6
WORSE MAX = 99.6 & 99.6 & 99.6
WORSE MIN = 99.6 & 99.6 & 99.6
BEST MEAN = 99.6 & 99.6 & 99.6
BEST MAX = 99.6 & 99.6 & 99.6
BEST MIN = 99.6 & 99.6 & 99.6
****************************************
Parsing node = A 
	 FSK - PDR = 99.5 & 99.6 & 99.6
	 OQPSK - PDR = 99.6 & 99.6 & 99.6
	 OFDM - PDR = 99.6 & 99.6 & 99.6
	 BES

<Figure size 432x288 with 0 Axes>